In [1]:
#The MIT License (MIT)

#Copyright (c) 2020 Juliana T.C. Marcos

#THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO
#THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE 
#AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF 
#CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

#This code use methods of the ParticleFilter (PF) class in order to track an object in a video. The PF uses as
#measurements provider a colour image segmentation technique.

#The particles are represented in red, the PF estimate of
#the tracked animal is represented in green and the animal's bb is represented in blue. 

In [2]:
#Import of useful librairies
import cv2
import math
import numpy as np
from skimage import measure
from ParticleFilter import ParticleFilter
import time


In [3]:
"""Some variables initialization """
#The total number of trials
Tot=1
#Lists for the Tot running outputs averages 
BB_avg=[]
anchor_avg=[]
xy_est_avg=[]
particles_avg=[]
#number of particles
n_particles=2000
#noise in sensors' measurements
meas_noise=0
font = cv2.FONT_HERSHEY_SIMPLEX
text_coord=(10,40)
t_size=0.8
t_thick=2
#Lists to contain the counters for each trial
cis_l=[]
n_resampl_l=[]

#This value was chosen according to a paper experiment
N_thresh=(2*n_particles)/30
#Video frame width and height
frame_width=1920
frame_height=1080
#Initialize variable for shifting the anchor update between first measurements and first estimations
anchor_shift=60

In [4]:
#Data for video cows
anchorS=(1250,350)
#This threshold helps to filter small contours
#It is however important to adapt it to the object scales in the videos
Area_thresh=0
#This is to choose the type of threshold (between cv2.THRESH_BINARY_INV 
#and THRESH_BINARY)
type_thr=cv2.THRESH_BINARY_INV
#Capture video where object tracking should be performed
path="./Outputs/"
videoIn_name ="./Inputs/cows.avi"
videoOut_name =path+"cows-pf-colour.avi"
template = cv2.imread('./Inputs/template2.png')
#number of particles
n_particles=2000
#std in the prediction of particles for the object's position
std=10
#Motion model's speed in x and y directions
v_x=0.01
v_y=0.01

In [5]:
start_time = time.time()

for num in range(Tot):
    #List for averaging running outputs
    BB_l=[]
    anchor_l=[]
    xy_est_l=[]
    particles_l=[]
    #Counters initialization
    cis=0
    it=0
    n_resampling=0
    anchor=anchorS
    #Initialization of measurements variables
    x_objMeasure,y_objMeasure=anchor[0],anchor[1]
    #BB variable initialization
    BB=0,0,0,0
    #Capture video where object tracking should be performed
    video = cv2.VideoCapture(videoIn_name)
    #Video output
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    video_output = cv2.VideoWriter(videoOut_name,fourcc,60,(frame_width,frame_height))
    #Particles Instantiation
    particles=ParticleFilter(frame_width,frame_height, n_particles)


    #This function uses open cv function to identify the center of the animal to track
    def sensors_measurements(template,img,anchor,mea_noise=3,kernel=3):

        #Compute the mean color of the template containing the animal to track color
        meanStdTemplate=cv2.meanStdDev(template)

        #Convert the color of the frame to work on and apply the GaussianBlur function in order to remove
        #Gaussian noise, smooth image and somrtimes highlight edges
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = cv2.GaussianBlur(img, (kernel, kernel), 0)

        #Binarize frame using the color mean of the animal in such a way that matching parts
        #of the frame will appear white and other parts will appear black
        ret,thresh = cv2.threshold(img,meanStdTemplate[0][1],255,type_thr)

        # find contours in the thresholded image. The if condition is used to avoid error that happens depending
        #on the python version used. The middle parameter is to only retrieve parent contours while the latter
        #is to avoid redundant points in contours.
        cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contours = cnts[0] if len(cnts) == 2 else cnts[1]

        #Compute the contours areas
        contoursAreas=[cv2.contourArea(c) for c in contours]

        #Compute the center and the BB of the contours
        contoursCenter=[]
        contoursBB=[]

        for c in range(len(contours)):
            M = cv2.moments(contours[c])
            #If the area of the current contour exists and is greater than a threshold for areas 
            if M["m00"] != 0 and contoursAreas[c]>Area_thresh:
                cX = int(M["m10"] / M["m00"])
                cY = int(M["m01"] / M["m00"])
                contoursCenter.append((c,cX,cY,contoursAreas[c]))
                contoursBB.append(cv2.boundingRect(contours[c]))

        #Compute the distance between each center and the anchor center in order to find 
        #the most suitable shape to track i.e the closest one to the anchor

        contoursAnchorDist=[math.sqrt((contoursCenter[i][1]-anchor[0])**2+\
        (contoursCenter[i][2]-anchor[1])**2) for i in range (len(contoursCenter))]

        #Save and return the coordinates of the most suitable center and its contours
        index=contoursAnchorDist.index(min(contoursAnchorDist))
        cX=contoursCenter[index][1]+np.random.standard_normal()*meas_noise
        cY=contoursCenter[index][2]+np.random.standard_normal()*meas_noise

        return cX,cY,contoursBB[index]

    #Loop through the entire video
    while (True):
        #Take the video and break it frame by frame
        _,frame=video.read()
        #Check if frames are captured
        if(_ == False ): break

        it+=1

        #Particles prediction update
        particles.particles_update(v_x,v_y,std,frame_width,frame_height)
        particles_l.append(particles.particles.copy())
        anchor_l.append(anchor)

        #Measurements
        x_objMeasure,y_objMeasure,BB = sensors_measurements(template,frame,anchor,meas_noise)
        #Use anchor to select the closest area and then the actual tracked target
        cis+=1
        #Save BB in a list
        BB_l.append(BB)
        
        #Update the particles weights with the new measurements (Object center)
        particles.weigth_update(x_objMeasure,y_objMeasure)

        #Estimation of object center position
        x_estimation,y_estimation=particles.position_estimation()

        #Save the x and y estimated in a list
        xy_est_l.append((x_estimation,y_estimation))

        #Draw the particles
        particles.draw_box_particles(frame,BB,x_estimation,y_estimation)
        
        #Draw the position estimation
        cv2.circle(frame,(x_estimation,y_estimation),5,[0,255,0],3)
       
        #Update the anchor with either the current measurements or the x and y estimates
        if it > anchor_shift:
            anchor=x_estimation,y_estimation
        else:
            anchor=x_objMeasure,y_objMeasure
        

        #Resample the particles
        if (particles.effective_particles() < N_thresh):
            n_resampling+=1
            particles.resampling()

        if (num==Tot-1):
            video_output.write(frame)

        #if it==50:
        #    break

    
    
    cis_l.append(cis)
    n_resampl_l.append(n_resampling)
    BB_avg.append(BB_l)
    xy_est_avg.append(xy_est_l)
    particles_avg.append(particles_l)
    anchor_avg.append(anchor_l)
    

In [6]:
prog_duration= time.time() - start_time
prog_duration

37.86215567588806

In [7]:
prog_duration/(60)

0.6310359279314677

In [8]:
prog_duration/(60*Tot)

0.6310359279314677

In [9]:
sum(n_resampl_l)/Tot

456.0

In [10]:
len(BB_l),len(xy_est_l),len(BB_avg),len(xy_est_avg),len(particles_avg),len(anchor_avg)

(980, 980, 1, 1, 1, 1)

In [11]:
sum(cis_l)/Tot

980.0

In [12]:
BB_l_avg=[] 
BB_avg=np.array(BB_avg)
BB_l_avg=np.sum(BB_avg,0)/Tot

In [13]:
xy_est_l_avg=[] 
xy_est_avg=np.array(xy_est_avg)
xy_est_l_avg=np.sum(xy_est_avg,0)/Tot
xy_est_l_avg

array([[1138.,  466.],
       [1228.,  395.],
       [1243.,  367.],
       ...,
       [1310.,  340.],
       [1311.,  341.],
       [1310.,  341.]])

In [14]:
particles_l_avg=[] 
particles_avg=np.array(particles_avg)
particles_l_avg=np.sum(particles_avg,0)/Tot

In [15]:
anchor_l_avg=[] 
anchor_avg=np.array(anchor_avg)
anchor_l_avg=np.sum(anchor_avg,0)/Tot
anchor_l_avg

array([[1250.,  350.],
       [1250.,  364.],
       [1252.,  364.],
       ...,
       [1310.,  340.],
       [1310.,  340.],
       [1311.,  341.]])

In [16]:
#xy_est_l

In [17]:
#BB_l

In [18]:
np.save(path+'xy_data1.npy',np.array(xy_est_l_avg))
np.save(path+'BB_data1.npy',np.array(BB_l_avg))
np.save(path+'part_data1.npy',np.array(particles_l_avg))
np.save(path+'anchor_data1.npy',np.array(anchor_l_avg))

In [19]:
len(BB_l_avg)

980

In [20]:
len(xy_est_l_avg)

980